In [8]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-135744")
exp = Experiment(workspace=ws, name="quick-starts-ws-135744")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135744
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-135744


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "hyperdriveml"
cpu_cluster = ComputeTarget(ws,cpu_cluster_name)

try:
       cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
       print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [10]:
%%writefile conda_dependencies.yml

dependencies:
  - python=3.6.2
  - scikit-learn
  - numpy
  - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


In [11]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [12]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps =  RandomParameterSampling({"max_iter" : choice(100,150,200,250),
                                "C" : choice(0.01,0.05,0.2,1,5,10)
                                })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target=cpu_cluster,                      
                      environment=sklearn_env)


# est = SKLearn(source_directory='./',               
#             compute_target=cpu_cluster,
#             entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs=4)

In [14]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
#experimnt = Experiment(ws,exp)
hd_run = exp.submit(hd_config)
hd_run.wait_for_completion(show_output=True)
RunDetails(hd_run).show()
### YOUR CODE HERE ###

RunId: HD_102bfbb1-b424-4ecf-b652-d763165f39cb
Web View: https://ml.azure.com/experiments/quick-starts-ws-135744/runs/HD_102bfbb1-b424-4ecf-b652-d763165f39cb?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-135744/workspaces/quick-starts-ws-135744

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-23T02:33:59.364487][API][INFO]Experiment created<END>\n"<START>[2021-01-23T02:34:00.3742014Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-23T02:33:59.888527][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-23T02:34:00.105214][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_102bfbb1-b424-4ecf-b652-d763165f39cb
Web View: https://ml.azure.com/experiments/quick-starts-ws-135744/runs/HD_102bfbb1-b424-4ecf-b652-d763165f39cb?wsid=/subsc

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [15]:
import joblib
# Get your best run and save the model from that run.
### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-135744,HD_102bfbb1-b424-4ecf-b652-d763165f39cb_6,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
print("Best run details", best_run.get_details())
print('\t')
print("Best run file names:", best_run.get_file_names())
model = best_run.register_model(model_name='bankmark_hyperdrive_best_run', model_path='./outputs/model.joblib')
print(model.name, model.id,model.version,sep='\t')

Best run details {'runId': 'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_6', 'target': 'hyperdriveml', 'status': 'Completed', 'startTimeUtc': '2021-01-23T02:36:17.868123Z', 'endTimeUtc': '2021-01-23T02:37:06.548207Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '91681480-2428-4f5e-9ac3-c3a53231716d', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.01', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'hyperdriveml', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'priority': None, 'credentialPassthrough': False, 'environment': {'name': 'sklearn-env', 'version': 'Autosave_2021-01-23T02:09:26Z_2cf7b965', 'python': {'i

In [17]:
hd_run.get_metrics()

{'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_0': {'Regularization Strength:': 10.0,
  'Max iterations:': 250,
  'Accuracy': 0.9077389984825494},
 'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_1': {'Regularization Strength:': 0.2,
  'Max iterations:': 200,
  'Accuracy': 0.9074355083459787},
 'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_10': {'Regularization Strength:': 0.01,
  'Max iterations:': 150,
  'Accuracy': 0.907587253414264},
 'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_11': {'Regularization Strength:': 1.0,
  'Max iterations:': 250,
  'Accuracy': 0.9077389984825494},
 'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_12': {'Regularization Strength:': 5.0,
  'Max iterations:': 100,
  'Accuracy': 0.908649468892261},
 'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_13': {'Regularization Strength:': 5.0,
  'Max iterations:': 150,
  'Accuracy': 0.9074355083459787},
 'HD_102bfbb1-b424-4ecf-b652-d763165f39cb_14': {'Regularization Strength:': 1.0,
  'Max iterations:': 200,
  'Accuracy': 0.907587253414264},
 'HD_102b

In [19]:
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.core.dataset import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(path=data_url)


In [20]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y
label = "y"

In [21]:
x.head()
x.to_csv("bankmarketing_train.csv")

In [24]:
from azureml.core.dataset import Dataset
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['bankmarketing_train.csv'],
                        target_path = './',
                        overwrite=True,
                        show_progress=True)
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, "./bankmarketing_train.csv")])

Uploading an estimated of 1 files
Uploading bankmarketing_train.csv
Uploaded bankmarketing_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [25]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name=label,
    n_cross_validations=3)

In [26]:
# Submit your automl run

### YOUR CODE HERE ###
#ws = Workspace.from_config()
#experiment = Experiment(ws, "exp")
run = exp.submit(automl_config, show_output=True)
run.wait_for_completion()

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_b9c5c826-f89b-4413-bf36-e3593994f452

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

{'runId': 'AutoML_b9c5c826-f89b-4413-bf36-e3593994f452',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-23T02:48:28.744211Z',
 'endTimeUtc': '2021-01-23T03:20:37.100799Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"quick-starts-ws-135744","subscription_id":"9e65f93e-bdd8-437b-b1e8-0647cd6098f7","resource_group":"aml-quickstarts-135744","workspace_name":"quick-starts-ws-135744","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classe

In [27]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: quick-starts-ws-135744,
Id: AutoML_b9c5c826-f89b-4413-bf36-e3593994f452_34,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                subsample=0.6436842105263159,
                                                                                                subsample_for_bin=200000,
                     

In [28]:
from azureml.interpret import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(best_run)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [34]:
client = ExplanationClient.from_run_id(ws, 'quick-starts-ws-135744', best_run.id)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [35]:
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [36]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

global importance values: [0.6214430958945745, 0.38255818244792145, 0.15170718927820648, 0.12880138313913514]
global importance names: ['duration', 'nr.employed', 'euribor3m', 'emp.var.rate']


In [37]:
model_name = best_run.properties['model_name']

script_file_name = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [ ]:
cpu_cluster.delete()